# OpenStreetMap extracts

**QuackOSM** exposes a simple API allowing user to download OSM extracts from multiple sources.

This example notebook shows how to: 
 - display available extracts,
 - download extract by name,
 - get extracts covering a given geometry. // TODO

<style>
div.jp-Cell-outputArea pre {
  overflow-y: auto;
  max-height: 50vh;
}
</style>

## Display available OSM extracts

Extracts have short name and a full name. Short name is just a current description of the region (eg. `Monaco`). Full name contains information about the extract source and all a whole nesting hierarchy (eg. `geofabrik_europe_monaco`).

By default, function for displaying those extracts shows short name. In these examples below full names have been used.

In [ ]:
from quackosm.osm_extracts import display_available_extracts, OsmExtractSource

### Geofabrik

In [ ]:
display_available_extracts(OsmExtractSource.geofabrik, use_full_names=True)

### OpenStreetMap FR

In [ ]:
display_available_extracts(OsmExtractSource.osm_fr, use_full_names=True)

### BBBike

In [ ]:
display_available_extracts(OsmExtractSource.bbbike, use_full_names=True)

## Download OSM extract by name

In [ ]:
from quackosm import convert_osm_extract_to_geodataframe

### Monaco extract from Geofabrik

Download data for the country of Monaco from the `Geofabrik` repository.

In [ ]:
convert_osm_extract_to_geodataframe("monaco", osm_extract_source="geofabrik")

### Vatican city

Download data for the Vatican city from any repository. Only `OpenStreetMap.fr` contains data for this query.

In [ ]:
convert_osm_extract_to_geodataframe("vatican_city")

### Query with multiple matches

Some extracts have the same name, or the same extract is available in multiple sources. Trying to get an extract by name with multiple matches will result in an error with list of extracts ids that can be used to get a specific one.

Getting a **Ceuta** region (sutonomous city of Spain bordering with Marocco).

Extract for this region is available both in Geofabrik and OSM fr sources.

In [ ]:
from rich import print as rprint
from rich.traceback import Traceback

from quackosm._exceptions import OsmExtractMultipleMatchesError

try:
    convert_osm_extract_to_geodataframe("ceuta")
except OsmExtractMultipleMatchesError as ex:
    rprint(Traceback.from_exception(type(ex), ex, None))

`OsmExtractMultipleMatchesError` has a property `matching_full_names` with a list of found extracts full names. It can be used programatically to access those values.

In [ ]:
from quackosm.osm_extracts import get_extract_by_query

matched_extracts = []

try:
    get_extract_by_query("ceuta")
except OsmExtractMultipleMatchesError as ex:
    for full_name in ex.matching_full_names:
        matched_extracts.append(get_extract_by_query(full_name))

matched_extracts

We can display both extracts extents on the map.

In [ ]:
import geopandas as gpd

gpd.GeoDataFrame(
    data=dict(id=[extract.id for extract in matched_extracts]),
    geometry=[extract.geometry for extract in matched_extracts],
    crs=4326,
).explore(column="id", tiles="CartoDB positron")

Let's download data for the extract from the OpenStreetMap.fr.

In [ ]:
convert_osm_extract_to_geodataframe("osmfr_africa_spain_ceuta")

### Query with zero matches

Sometimes query doesn't match any of the available extracts.
Function for finding the extracts, automatically detects close names in case this was a typo and suggests them to the user. Suggestions are sorted based on the closeness to the query.

In [ ]:
from quackosm._exceptions import OsmExtractZeroMatchesError

try:
    convert_osm_extract_to_geodataframe("Gremany")
except OsmExtractZeroMatchesError as ex:
    rprint(Traceback.from_exception(type(ex), ex, None))

`OsmExtractZeroMatchesError` has a property `matching_full_names` with a list of suggested matching names. It can be used programatically to access those values.

In [ ]:
closest_matching_extract = None

try:
    get_extract_by_query("pland")
except OsmExtractZeroMatchesError as ex:
    rprint(Traceback.from_exception(type(ex), ex, None))
    closest_matching_extract = get_extract_by_query(ex.matching_full_names[0])

closest_matching_extract

Sometimes query can yield zero closest matches.

In [ ]:
try:
    get_extract_by_query("totally_nonexistent_osm_extract")
except OsmExtractZeroMatchesError as ex:
    rprint(Traceback.from_exception(type(ex), ex, None))